In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import lines
from collections import defaultdict, deque, Counter
import heapq

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time
from FctUtil import Node,hashable_array


ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
class EightPuzzle:
    def __init__(self):
        #Initial state (we can initialise our matric with it)
        self.initial_state=np.array([[7,2,4],[5,-1,6],[8,3,1]])
        self.goal_state=np.array([[-1,1,2],[3,4,5],[6,7,8]])
    @property
    def Initial_State(self):
        return self.initial_state

    @property
    def Goal_State(self):
        return self.goal_state

    #By actions we mean that the empty(here -1) case go up or down ...
    def move(self,action,state):
        new_state = state.copy()
        row, col =np.where(state==-1)
        if(action=="up"):
            new_state[row, col], new_state[row - 1, col] = new_state[row - 1, col], new_state[row, col]
        elif(action=="down"):
            new_state[row, col], new_state[row + 1, col] = new_state[row + 1, col], new_state[row, col]
        elif(action=="left"):
            new_state[row, col], new_state[row, col -1] = new_state[row, col-1], new_state[row, col]
        elif(action=="right"):
            new_state[row, col], new_state[row, col +1] = new_state[row, col+1], new_state[row, col]

        return new_state

    def move_allowed(self,state):
        tab_actions = []  # List to tell which actions are allowed from your node
        row, col = np.where(state == -1)
        row, col = row[0], col[0]  # obtain the scalar indices
        if row != 0:tab_actions.append("up")
        if col != 2:tab_actions.append("right")
        if row != 2:tab_actions.append("down")
        if col != 0:tab_actions.append("left")
        return np.array(tab_actions)  # Transform the list in a numpy array

    # Fonction that tells you if you reach the goal or not
    def GoalReach(self,state):
        #This is the Goal Matrix
        return np.array_equal(state,self.goal_state)

    def result(self,state,move):
        return self.move(move, state)

In [ ]:
import heapq
#https://docs.python.org/fr/3/library/heapq.html
def best_first_graph_search(problem, f, heuristics):
    """We has f and heuristics to do this case : 
        if f Not None and heuristics = None we are using UCS
        if f None and heuristics Not None we are using bfs
        IF f Not None and heuristics Not None we are using A*
    """
    
    # Initialize the priority queue 
    frontier = []
    
    # Initialize the first node
    curent = Node(problem.Initial_State)
    
    # Add the initial node in the priority queue using heapq
    heapq.heappush(frontier, curent)
    
    # Use a set to keep track of explored states
    explored = set()
    
    while frontier:
        # Extract the node with the lowest f score
        curent = heapq.heappop(frontier)
        
        curent_state_hash = hashable_array(curent.state)
    
        # Check if the current state is the goal
        if problem.GoalReach(curent.state):
            return curent, explored
        
        # Add the current state to the explored set
        explored.add(curent_state_hash)  # Convert to tuple to make it hashable
        
        # Iterate over the children of the current node
        for child in curent.expand(problem, f, heuristics):
            child_state_hash = hashable_array(child.state)  # Convert to tuple

            # If the child has not been explored and is not in the frontier
            if child_state_hash not in explored: #and child not in frontier 
                """I didn't verify if it's i the frontier or not because even if I explored more nodes it's longer to compare each time to each node in the frontier"""
                heapq.heappush(frontier, child)

    return None  # Returns None if no path is found


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def afficher_matrices_noir_blanc(matrices):
    """
    Affiche une liste de matrices 3x3 les unes après les autres, sans couleur,
    avec les valeurs des matrices affichées au centre de chaque case.
    
    :param matrices: Liste de matrices NumPy de dimension 3x3
    """
    # Nombre total de matrices
    n = len(matrices)

    # Créer un subplot pour chaque matrice
    fig, axes = plt.subplots(1, n, figsize=(4 * n, 4))
    
    # S'assurer que `axes` est toujours une liste même si on a une seule matrice
    if n == 1:
        axes = [axes]
    
    # Boucle à travers les matrices et affichage
    for i, mat in enumerate(matrices):
        ax = axes[i]
        ax.matshow(np.ones_like(mat), cmap='gray_r')  # Fond en noir et blanc

        # Afficher les valeurs des matrices au centre de chaque cellule
        for (j, k), val in np.ndenumerate(mat):
            ax.text(k, j, f'{val}', ha='center', va='center', fontsize=16)
        
        # Configurer les axes
        ax.set_xticks([])  # Supprimer les graduations des axes x
        ax.set_yticks([])  # Supprimer les graduations des axes y
        ax.set_title(f" {i+1}", fontsize=16)
    
    plt.tight_layout()
    plt.show()


In [ ]:
"""
Funtion to print the result for a search 
"""
def PrintResult(problem,node,chemin):
    # Path with all the nodes explored
    print(f"There are {len(chemin)} nodes explored\n")
    parcours = node.path()
    solution=node.solution()
    print(f"There are {len(parcours)} moves to reach our goal:")
    print(f"Start : \n{problem.initial_state}\n")
    for n in range(len(parcours)):
        print(f"{solution[n]} :\n{parcours[n].state}\n")

In [ ]:
"""
Fuction to compare 2 search
"""
def Compare(problem,optimal,node1,chemin1,node2,chemin2,nom1,nom2):
    # Performance
    len_chemin1=len(chemin1)
    len_chemin2=len(chemin2)
    print(f"There are {len_chemin1} nodes explored for the {nom1}")
    print(f"There are {len_chemin2} nodes explored for the {nom2}\n")
    if(len_chemin1>len_chemin2):print(f"Thus {nom2} is faster than {nom1}\n")
    elif(len_chemin1<len_chemin2):print(f"Thus {nom1} is faster than {nom2}\n")
    else:print(f"Thus {nom2} and {nom1} have the same speed\n")

    #Optimality
    parcours1=node1.path()
    parcours2=node2.path()
    if(len(parcours1)==optimal and len(parcours2)==optimal):print(f"{nom1} and {nom2} are both optimal")
    elif(len(parcours1)==optimal and len(parcours2)!=optimal):print(f"{nom1} is optimal but {nom2} is not")
    elif(len(parcours2)==optimal and len(parcours1)!=optimal):print(f"{nom2} is optimal but {nom1} is not")
    else : print(f"{nom1} and {nom2} are both not optimal")
    print("\n")

In [ ]:
"""
Implementation of the function adding 1 to the child's past cost of the current node
"""
def f(node_path_cost):
    return node_path_cost+1

In [ ]:
def uniform_cost_search(problem):
    node, chemin = best_first_graph_search(problem,f,None)
    PrintResult(problem,node,chemin)
    return node,chemin
                        



PROBLEM INITIALISATION

In [ ]:
#Initialize the problem
problem = EightPuzzle()

In [ ]:
nodeUCS,cheminUCS =uniform_cost_search(problem)
# Takes 1 min

There are 169856 nodes explored

There are 26 moves to reach our goal:
Start : 
[[ 7  2  4]
 [ 5 -1  6]
 [ 8  3  1]]

left :
[[ 7  2  4]
 [-1  5  6]
 [ 8  3  1]]

up :
[[-1  2  4]
 [ 7  5  6]
 [ 8  3  1]]

right :
[[ 2 -1  4]
 [ 7  5  6]
 [ 8  3  1]]

down :
[[ 2  5  4]
 [ 7 -1  6]
 [ 8  3  1]]

down :
[[ 2  5  4]
 [ 7  3  6]
 [ 8 -1  1]]

left :
[[ 2  5  4]
 [ 7  3  6]
 [-1  8  1]]

up :
[[ 2  5  4]
 [-1  3  6]
 [ 7  8  1]]

right :
[[ 2  5  4]
 [ 3 -1  6]
 [ 7  8  1]]

right :
[[ 2  5  4]
 [ 3  6 -1]
 [ 7  8  1]]

up :
[[ 2  5 -1]
 [ 3  6  4]
 [ 7  8  1]]

left :
[[ 2 -1  5]
 [ 3  6  4]
 [ 7  8  1]]

left :
[[-1  2  5]
 [ 3  6  4]
 [ 7  8  1]]

down :
[[ 3  2  5]
 [-1  6  4]
 [ 7  8  1]]

right :
[[ 3  2  5]
 [ 6 -1  4]
 [ 7  8  1]]

right :
[[ 3  2  5]
 [ 6  4 -1]
 [ 7  8  1]]

down :
[[ 3  2  5]
 [ 6  4  1]
 [ 7  8 -1]]

left :
[[ 3  2  5]
 [ 6  4  1]
 [ 7 -1  8]]

left :
[[ 3  2  5]
 [ 6  4  1]
 [-1  7  8]]

up :
[[ 3  2  5]
 [-1  4  1]
 [ 6  7  8]]

right :
[[ 3  2  5]
 [ 4 -1  1

Implémentation des heuristics

In [ ]:
#Counts the number of tiles that are not in their goal position
def h1(state, goal_state):
    return np.sum(state != goal_state) - 1  # Subtract 1 for the empty tile (-1)

In [ ]:
#Measures the total distance each tile needs to move to reach its goal position.

def h2(state, goal_state):
    distance = 0
    
    for i in range(3):  
        for j in range(3):
            value = state[i, j]
            if value != -1:  # Let’s assume that -1 represents the empty space.
                # Find the coordinates of the value in the goal state.
                goal_row, goal_col = np.where(goal_state == value)
                # Compute the Manhattan distance
                distance += abs(i - goal_row) + abs(j - goal_col)
    
    return distance




In [ ]:
def using_h1(problem):
    node, chemin = best_first_graph_search(problem,None, lambda state: h1(state,goal_state=problem.goal_state))
    PrintResult(problem,node,chemin)  
    return node,chemin
    

In [ ]:
nodeBFS_h1,cheminBFS_h1=using_h1(problem)

There are 464 nodes explored

There are 72 moves to reach our goal:
Start : 
[[ 7  2  4]
 [ 5 -1  6]
 [ 8  3  1]]

left :
[[ 7  2  4]
 [-1  5  6]
 [ 8  3  1]]

up :
[[-1  2  4]
 [ 7  5  6]
 [ 8  3  1]]

right :
[[ 2 -1  4]
 [ 7  5  6]
 [ 8  3  1]]

right :
[[ 2  4 -1]
 [ 7  5  6]
 [ 8  3  1]]

down :
[[ 2  4  6]
 [ 7  5 -1]
 [ 8  3  1]]

left :
[[ 2  4  6]
 [ 7 -1  5]
 [ 8  3  1]]

up :
[[ 2 -1  6]
 [ 7  4  5]
 [ 8  3  1]]

left :
[[-1  2  6]
 [ 7  4  5]
 [ 8  3  1]]

down :
[[ 7  2  6]
 [-1  4  5]
 [ 8  3  1]]

down :
[[ 7  2  6]
 [ 8  4  5]
 [-1  3  1]]

right :
[[ 7  2  6]
 [ 8  4  5]
 [ 3 -1  1]]

right :
[[ 7  2  6]
 [ 8  4  5]
 [ 3  1 -1]]

up :
[[ 7  2  6]
 [ 8  4 -1]
 [ 3  1  5]]

up :
[[ 7  2 -1]
 [ 8  4  6]
 [ 3  1  5]]

left :
[[ 7 -1  2]
 [ 8  4  6]
 [ 3  1  5]]

left :
[[-1  7  2]
 [ 8  4  6]
 [ 3  1  5]]

down :
[[ 8  7  2]
 [-1  4  6]
 [ 3  1  5]]

down :
[[ 8  7  2]
 [ 3  4  6]
 [-1  1  5]]

right :
[[ 8  7  2]
 [ 3  4  6]
 [ 1 -1  5]]

up :
[[ 8  7  2]
 [ 3 -1  6]
 [ 1

In [ ]:
def using_h2(problem):
    node, chemin = best_first_graph_search(problem,None, lambda state: h2(state,goal_state=problem.goal_state))
    PrintResult(problem,node,chemin)
    return node,chemin

In [ ]:
nodeBFS_h2,cheminBFS_h2=using_h2(problem)


There are 222 nodes explored

There are 54 moves to reach our goal:
Start : 
[[ 7  2  4]
 [ 5 -1  6]
 [ 8  3  1]]

right :
[[ 7  2  4]
 [ 5  6 -1]
 [ 8  3  1]]

up :
[[ 7  2 -1]
 [ 5  6  4]
 [ 8  3  1]]

left :
[[ 7 -1  2]
 [ 5  6  4]
 [ 8  3  1]]

left :
[[-1  7  2]
 [ 5  6  4]
 [ 8  3  1]]

down :
[[ 5  7  2]
 [-1  6  4]
 [ 8  3  1]]

right :
[[ 5  7  2]
 [ 6 -1  4]
 [ 8  3  1]]

down :
[[ 5  7  2]
 [ 6  3  4]
 [ 8 -1  1]]

left :
[[ 5  7  2]
 [ 6  3  4]
 [-1  8  1]]

up :
[[ 5  7  2]
 [-1  3  4]
 [ 6  8  1]]

right :
[[ 5  7  2]
 [ 3 -1  4]
 [ 6  8  1]]

right :
[[ 5  7  2]
 [ 3  4 -1]
 [ 6  8  1]]

down :
[[ 5  7  2]
 [ 3  4  1]
 [ 6  8 -1]]

left :
[[ 5  7  2]
 [ 3  4  1]
 [ 6 -1  8]]

up :
[[ 5  7  2]
 [ 3 -1  1]
 [ 6  4  8]]

up :
[[ 5 -1  2]
 [ 3  7  1]
 [ 6  4  8]]

left :
[[-1  5  2]
 [ 3  7  1]
 [ 6  4  8]]

down :
[[ 3  5  2]
 [-1  7  1]
 [ 6  4  8]]

right :
[[ 3  5  2]
 [ 7 -1  1]
 [ 6  4  8]]

right :
[[ 3  5  2]
 [ 7  1 -1]
 [ 6  4  8]]

up :
[[ 3  5 -1]
 [ 7  1  2]
 [ 

In [ ]:
#= number of tiles not in the correct row + number of tiles not in the correct column.
def h3(state, goal_state):
    distance =0
    for i in range(3):  
        for j in range(3):
            value = state[i, j]
            if value != -1:  # Let’s assume that -1 represents the empty space.
                # Find the coordinates of the value in the goal state.
                goal_row, goal_col = np.where(goal_state == value)
                # Compute the distance
                if(goal_row!=i):distance+=1
                if(goal_col!=j):distance+=1
    return distance
    

In [ ]:
def using_h3(problem):
    node, chemin = best_first_graph_search(problem,None, lambda state: h3(state,goal_state=problem.goal_state))
    PrintResult(problem,node,chemin)
    return node,chemin

In [ ]:
nodeBFS_h3,cheminBFS_h3=using_h3(problem)

There are 433 nodes explored

There are 98 moves to reach our goal:
Start : 
[[ 7  2  4]
 [ 5 -1  6]
 [ 8  3  1]]

down :
[[ 7  2  4]
 [ 5  3  6]
 [ 8 -1  1]]

right :
[[ 7  2  4]
 [ 5  3  6]
 [ 8  1 -1]]

up :
[[ 7  2  4]
 [ 5  3 -1]
 [ 8  1  6]]

up :
[[ 7  2 -1]
 [ 5  3  4]
 [ 8  1  6]]

left :
[[ 7 -1  2]
 [ 5  3  4]
 [ 8  1  6]]

left :
[[-1  7  2]
 [ 5  3  4]
 [ 8  1  6]]

down :
[[ 5  7  2]
 [-1  3  4]
 [ 8  1  6]]

right :
[[ 5  7  2]
 [ 3 -1  4]
 [ 8  1  6]]

down :
[[ 5  7  2]
 [ 3  1  4]
 [ 8 -1  6]]

left :
[[ 5  7  2]
 [ 3  1  4]
 [-1  8  6]]

up :
[[ 5  7  2]
 [-1  1  4]
 [ 3  8  6]]

up :
[[-1  7  2]
 [ 5  1  4]
 [ 3  8  6]]

right :
[[ 7 -1  2]
 [ 5  1  4]
 [ 3  8  6]]

down :
[[ 7  1  2]
 [ 5 -1  4]
 [ 3  8  6]]

right :
[[ 7  1  2]
 [ 5  4 -1]
 [ 3  8  6]]

down :
[[ 7  1  2]
 [ 5  4  6]
 [ 3  8 -1]]

left :
[[ 7  1  2]
 [ 5  4  6]
 [ 3 -1  8]]

up :
[[ 7  1  2]
 [ 5 -1  6]
 [ 3  4  8]]

left :
[[ 7  1  2]
 [-1  5  6]
 [ 3  4  8]]

down :
[[ 7  1  2]
 [ 3  5  6]
 [-1 

In [ ]:
def A_star(problem,f,h):
    node, chemin = best_first_graph_search(problem,f, lambda node: h(node,goal_state=problem.goal_state))
    PrintResult(problem,node,chemin)
    # afficher_matrices_noir_blanc(node.path()) à modifier
    return node,chemin


In [ ]:
#Using h1
nodeA_star_h1,cheminA_star_h1=A_star(problem,f,h1)

There are 42254 nodes explored

There are 26 moves to reach our goal:
Start : 
[[ 7  2  4]
 [ 5 -1  6]
 [ 8  3  1]]

left :
[[ 7  2  4]
 [-1  5  6]
 [ 8  3  1]]

up :
[[-1  2  4]
 [ 7  5  6]
 [ 8  3  1]]

right :
[[ 2 -1  4]
 [ 7  5  6]
 [ 8  3  1]]

down :
[[ 2  5  4]
 [ 7 -1  6]
 [ 8  3  1]]

down :
[[ 2  5  4]
 [ 7  3  6]
 [ 8 -1  1]]

left :
[[ 2  5  4]
 [ 7  3  6]
 [-1  8  1]]

up :
[[ 2  5  4]
 [-1  3  6]
 [ 7  8  1]]

right :
[[ 2  5  4]
 [ 3 -1  6]
 [ 7  8  1]]

right :
[[ 2  5  4]
 [ 3  6 -1]
 [ 7  8  1]]

up :
[[ 2  5 -1]
 [ 3  6  4]
 [ 7  8  1]]

left :
[[ 2 -1  5]
 [ 3  6  4]
 [ 7  8  1]]

left :
[[-1  2  5]
 [ 3  6  4]
 [ 7  8  1]]

down :
[[ 3  2  5]
 [-1  6  4]
 [ 7  8  1]]

right :
[[ 3  2  5]
 [ 6 -1  4]
 [ 7  8  1]]

right :
[[ 3  2  5]
 [ 6  4 -1]
 [ 7  8  1]]

down :
[[ 3  2  5]
 [ 6  4  1]
 [ 7  8 -1]]

left :
[[ 3  2  5]
 [ 6  4  1]
 [ 7 -1  8]]

up :
[[ 3  2  5]
 [ 6 -1  1]
 [ 7  4  8]]

right :
[[ 3  2  5]
 [ 6  1 -1]
 [ 7  4  8]]

up :
[[ 3  2 -1]
 [ 6  1  5]
 

In [ ]:
#Using h2
nodeA_star_h2,cheminA_star_h2=A_star(problem,f,h2)

There are 2761 nodes explored

There are 26 moves to reach our goal:
Start : 
[[ 7  2  4]
 [ 5 -1  6]
 [ 8  3  1]]

left :
[[ 7  2  4]
 [-1  5  6]
 [ 8  3  1]]

up :
[[-1  2  4]
 [ 7  5  6]
 [ 8  3  1]]

right :
[[ 2 -1  4]
 [ 7  5  6]
 [ 8  3  1]]

down :
[[ 2  5  4]
 [ 7 -1  6]
 [ 8  3  1]]

right :
[[ 2  5  4]
 [ 7  6 -1]
 [ 8  3  1]]

down :
[[ 2  5  4]
 [ 7  6  1]
 [ 8  3 -1]]

left :
[[ 2  5  4]
 [ 7  6  1]
 [ 8 -1  3]]

left :
[[ 2  5  4]
 [ 7  6  1]
 [-1  8  3]]

up :
[[ 2  5  4]
 [-1  6  1]
 [ 7  8  3]]

right :
[[ 2  5  4]
 [ 6 -1  1]
 [ 7  8  3]]

right :
[[ 2  5  4]
 [ 6  1 -1]
 [ 7  8  3]]

down :
[[ 2  5  4]
 [ 6  1  3]
 [ 7  8 -1]]

left :
[[ 2  5  4]
 [ 6  1  3]
 [ 7 -1  8]]

left :
[[ 2  5  4]
 [ 6  1  3]
 [-1  7  8]]

up :
[[ 2  5  4]
 [-1  1  3]
 [ 6  7  8]]

right :
[[ 2  5  4]
 [ 1 -1  3]
 [ 6  7  8]]

right :
[[ 2  5  4]
 [ 1  3 -1]
 [ 6  7  8]]

up :
[[ 2  5 -1]
 [ 1  3  4]
 [ 6  7  8]]

left :
[[ 2 -1  5]
 [ 1  3  4]
 [ 6  7  8]]

left :
[[-1  2  5]
 [ 1  3  4]


In [ ]:
# We knwow that UCS is optimal with weight > 0
Optimal = len(nodeUCS.path())
nom1="Uniform Cost Search"
nom2="A* using h1"
nom3="A* using h2"
nom4 = "BFS using h1"
nom5 = 'BFS using h2'



In [ ]:
# UCS and A* with h1
Compare(problem,Optimal,nodeUCS,cheminUCS,nodeA_star_h1,cheminA_star_h1,nom1,nom2)

There are 169856 nodes explored for the Uniform Cost Search
There are42254 nodes explored for the A* using h1

Thus A* using h1 is faster than Uniform Cost Search

Uniform Cost Search and A* using h1 are both optimal




In [ ]:
# UCS and A* with h2
Compare(problem,Optimal,nodeUCS,cheminUCS,nodeA_star_h2,cheminA_star_h2,nom1,nom3)

There are 169856 nodes explored for the Uniform Cost Search
There are2761 nodes explored for the A* using h2

Thus A* using h2 is faster than Uniform Cost Search

Uniform Cost Search and A* using h2 are both optimal




In [ ]:
#A* with h1 and BFS with h1
Compare(problem,Optimal,nodeBFS_h1,cheminBFS_h1,nodeA_star_h1,cheminA_star_h1,nom4,nom2)

There are 464 nodes explored for the BFS using h1
There are42254 nodes explored for the A* using h1

Thus BFS using h1 is faster than A* using h1

A* using h1 is optimal but BFS using h1 is not




In [ ]:
#A* with h2 and BFS with h2
Compare(problem,Optimal,nodeBFS_h2,cheminBFS_h2,nodeA_star_h2,cheminA_star_h2,nom5,nom3)

There are 222 nodes explored for the BFS using h2
There are2761 nodes explored for the A* using h2

Thus BFS using h2 is faster than A* using h2

A* using h2 is optimal but BFS using h2 is not


